In [1]:
import timeit
import os
import logging
logger = logging.getLogger(__name__)

In [2]:
path = "C:/WILDRETTER DATEN/2020_Easy/700101_0000_Fotosession2"
#path = "G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras/FLIR/FLIR_Vue_640_20200517_110530"
#path = "E:/2014/Vabene++/Flug119_2014-07-11_08-07_Germering_Foto_1s/GoPro"
filelist = []
for root,folders, files in os.walk(path):
        for f in files:
            base, ext = os.path.splitext(f)
            if ext.lower() not in [".jpg",".jpeg"]: continue
            filelist.append(os.path.join(root,f))    
print (len(filelist))

155


In [3]:
import re

MARKER = {
b"\xff\xe0":"APP0" , # JFIF APP0 segment marker
b"\xff\xe1":"APP1" ,
b"\xff\xe2":"APP2" ,
b"\xff\xe3":"APP3" ,
b"\xff\xe4":"APP4" ,
b"\xff\xe5":"APP5" ,
b"\xff\xe6":"APP6" ,
b"\xff\xe7":"APP7" ,
b"\xff\xe8":"APP8" ,
b"\xff\xe9":"APP9" ,
b"\xff\xea":"APP10",
b"\xff\xeb":"APP11",
b"\xff\xec":"APP12",
b"\xff\xed":"APP13",
b"\xff\xee":"APP14",
b"\xff\xef":"APP15",
b"\xff\xc0":"SOF0" , # Start Of Frame (baseline JPEG) 
b"\xff\xc1":"SOF1" , # Start Of Frame (baseline JPEG) 
b"\xff\xc2":"SOF2" ,
b"\xff\xc3":"SOF3" ,
b"\xff\xc4":"SOF4" ,
b"\xff\xc5":"SOF5" ,
b"\xff\xc6":"SOF6" ,
b"\xff\xc7":"SOF7" ,
b"\xff\xc9":"SOF9" ,
b"\xff\xca":"SOF10",
b"\xff\xcb":"SOF11",
b"\xff\xcd":"SOF13",
b"\xff\xce":"SOF14",
b"\xff\xcf":"SOF15",
b"\xff\xc4":"DHT"  , # Define Huffman Table
b"\xff\xdb":"DQT"  , # Define Quantization Table
b"\xff\xc8":"JPG"  ,
b"\xff\xf0":"JPG0" ,
b"\xff\xfd":"JPG13",
b"\xff\xcc":"DAC"  , # Define Arithmetic Table, usually unsupport 
b"\xff\xdc":"DNL"  ,
b"\xff\xdd":"DRI"  , # Define Restart Interval
b"\xff\xde":"DHP"  ,
b"\xff\xdf":"EXP"  ,
b"\xff\xd0":"RST0" ,  # RSTn are used for resync, may be ignored
b"\xff\xd1":"RST1" ,
b"\xff\xd2":"RST2" ,
b"\xff\xd3":"RST3" ,
b"\xff\xd4":"RST4" ,
b"\xff\xd5":"RST5" ,
b"\xff\xd6":"RST6" ,
b"\xff\xd7":"RST7" ,
    
b"\xff\x01":"TEM"  ,
b"\xff\xfe":"COM"  } # Comment
# ffd9 is end of image
           
    

TYPES = {1: {"id":1, "name":"Byte","format":"B","size":1},
        2: {"id":2,  "name":"Ascii","format":"s","size":1},
        3: {"id":3,  "name":"Short","format":"H","size":2},
        4: {"id":4,  "name":"Long","format":"L","size":4},
        5: {"id":5,  "name":"Rational","format":"LL","size":8},
        6: {"id":6,  "name":"SByte","format":"b","size":1},
        7: {"id":7,  "name":"Undefined","format":"B","size":1}, # B
        8: {"id":8,  "name":"SShort","format":"h","size":2},
        9: {"id":9,  "name":"SLong","format":"l","size":4},
        10: {"id":10,"name":"SRational","format":"ll","size":8},
        11: {"id":11,"name":"Float","format":"f","size":4},
        12: {"id":12,"name":"Double","format":"d","size":8},
        }


ENDIAN = {b"II*\x00":{"name":"Little Endian","format":"<","data":b"II*\x00","shortname":"little"},
          b"MM*\x00":{"name":"Big Endian","format":">","data":b"MM*\x00","shortname":"big"}}

IFD_TAG_SIZE = 12 # HHII (TagID (short), Type (short), Count (int),ValueOffset (int) )
NEXT_IFD_OFFSET_SIZE = 4
NUM_OF_INTEROP_SIZE = 2

MARKER_2BYTE = {
    b"\xff\xd8":"SOI"  ,  # Start of Image
    b"\xff\xd9":"EOI"  ,  # End of Image
    b"\xff\xd0":"RST0" ,  # RSTn are used for resync, may be ignored
    b"\xff\xd1":"RST1" ,
    b"\xff\xd2":"RST2" ,
    b"\xff\xd3":"RST3" ,
    b"\xff\xd4":"RST4" ,
    b"\xff\xd5":"RST5" ,
    b"\xff\xd6":"RST6" ,
    b"\xff\xd7":"RST7" ,
   # b"\xff\xda":"SOS"  ,  # Start of Scan
    b"\xff\x01":"TEM"  ,
    }

MARKER_2BYTE_REV = {y:x for x,y in MARKER_2BYTE.items()}

MARKER_REV = {y:x for x,y in MARKER.items()}


class JFIF(object):
    
    def __find_segments__(self,data):
        """
        A Jpeg file is normally structured as JFIF.
        JFIF has Segments (see MARKER)
        
        returns the complete File as a Segments-List
        """
        pattern = bytearray(b"..|".join(MARKER.keys()))
        pattern.extend(b"..|")
        pattern.extend(b"|".join(MARKER_2BYTE.keys()))
        cpattern = re.compile(bytes(pattern))
        segments = []
        id = 0
        parent = 0
        seemless = True
        for m in cpattern.finditer(data):
            if len(m.group())>=4: 
                seg_length = 256 * m.group()[2] + m.group()[3]
                segment = MARKER[m.group()[:-2]]
            else:
                seg_length = 0
                try:
                    segment = MARKER[m.group()]
                except KeyError:
                    segment = MARKER_2BYTE[m.group()]
                except: 
                    logging.error("find JFIF Segment failed",exc_info=True)
            if segment in ["APP0","APP1"]:
                seg_type = data[m.start()+4:m.start()+8]
            else:
                seg_type = ''
                        
            parent,level = next(((item["id"],item["level"]+1) 
                                for item in reversed(segments) 
                                    if item["end"] > m.start()), (0,0))
            
          # if len(segments)>0:
          #      seemless = True if m.start() == segments[-1]["end"] else False
                
            segments.append({"id":id,
                "segment": segment,
                "pos":m.start(),
                "end" : m.start()+seg_length+2,
                "type": seg_type,             
                "parent": parent, 
                "level": level, 
           #     "seemless": seemless,             
                "data": data[m.start() : m.start() + seg_length+2]   })
            id += 1
        return segments

    def find_segment(self,segmentname):
        idx = next((k for k,i in enumerate(self.segments) if i["segment"]== segmentname and i["level"]==0),None)
        if idx==None:
            logging.warning("Segment %s not found in this Jpeg." %segmentname)
        return idx
    
    def find_segment_type(self,stype, onlytoplevel = True, quiet=False):
        if onlytoplevel:
            idx = next((k for k,i in enumerate(self.segments) if i["type"]== stype and i["level"]==0),None)
        else:
            idx = next((k for k,i in enumerate(self.segments) if i["type"]== stype),None)
        if idx==None:
            if not quiet:
                logging.warning("Segment %s not found in this Jpeg." %stype)
        return idx    
    
    def get_segment_data(self,name="APP4",level=0):
        for s in self.segments:
            if s["segment"]==name and s["level"]==level:
                return s["data"]
        return b''


    def translate_tags(self,rawdata,tagstruct):    
        params = {}
        for i in tagstruct:
            val = struct.Struct("<"+i[2]).unpack_from(rawdata,i[0])
            #print(i[2])
            if "s" in i[2]:
                val = val[0].strip(b"\x00")
            name = i[1]
            params[name]=val
        return params    

In [4]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1: return
        yield start
        start += len(sub) # use start += 1 to find overlapping matches
    
def find_segments_str(data):
    mylst = []
    for i in find_all(data ,b"\xFF"):
        mylst.append((data[i:i+4],i))
    #return mylst
    
def find_segments_re(data):
        pattern = bytearray(b"..|".join(MARKER.keys()))
        pattern.extend(b"..|")
        pattern.extend(b"|".join(MARKER_2BYTE.keys()))
        cpattern = re.compile(bytes(pattern))
        segments = []
        id = 0
        parent = 0
        seemless = True
        for m in cpattern.finditer(data):
            if len(m.group())>=4: 
                seg_length = 256 * m.group()[2] + m.group()[3]
                segment = MARKER[m.group()[:-2]]
            else:
                seg_length = 0
                try:
                    segment = MARKER[m.group()]
                except KeyError:
                    segment = MARKER_2BYTE[m.group()]
                except: 
                    logging.error("find JFIF Segment failed",exc_info=True)
            if segment in ["APP0","APP1"]:
                seg_type = data[m.start()+4:m.start()+8]
            else:
                seg_type = ''
                        
            parent,level = next(((item["id"],item["level"]+1) 
                                for item in reversed(segments) 
                                    if item["end"] > m.start()), (0,0))
            
            segments.append({"id":id,
                "segment": segment,
                "pos":m.start(),
                "end" : m.start()+seg_length+2,
                "type": seg_type,             
                "parent": parent, 
                "level": level, 
           #     "seemless": seemless,             
                "data": data[m.start() : m.start() + seg_length+2]   })
            id += 1
        return segments


def find_segments_str2(data):
    n = 0
    mylst = []
    pos = 0
    length = 0
    offset = 2
    inside_sos = False
    while True:
        prepos = pos
        pos = data.find(b"\xFF",pos)
        if pos == -1: break
        marker = data[pos:pos+2]
        if marker in MARKER.keys():
            name = MARKER[marker]
            #print(name)
            length = data[pos+2:pos+4]
            pos = pos + 256 * length[0] + length[1] + offset
            inside_sos = False
            n = n +1
            mylst.append((n,prepos,name,pos))     
        elif marker in MARKER_2BYTE.keys():
            name = MARKER_2BYTE[marker]
            #print(name)
            pos = pos + offset
            inside_sos = False
            n = n +1
            mylst.append((n,prepos,name,0))     
        elif marker == b"\xff\x00": # VALID DATA
            #print("FF 00")
            pos = pos + 1
        elif marker == b"\xff\xda": # Start of Scan has not a known length
            inside_sos = True
            #print("SOS")
            pos = pos + 1
        else:
            pos = pos + 1
            #print("NOTHING")
    return mylst

def find_main_segments(data):
    mylst = []
    pos = 0
    offset = 2
    while True:
        prepos = pos
        pos = data.find(b"\xFF",pos)
        if pos == -1: break
        marker = data[pos:pos+2]
        if marker in MARKER.keys():
            name = MARKER[marker]
            length = data[pos+2:pos+4]
            pos = pos + 256 * length[0] + length[1] + offset
            mylst.append((prepos,name,pos))     
        elif marker in MARKER_2BYTE.keys():
            name = MARKER_2BYTE[marker]
            pos = pos + offset
            mylst.append((prepos,name,pos)) 
        else:
            pos = pos + 1
            if marker == b"\xff\xda": # Start of Scan has not a known length
                mylst.append((prepos,"SOS",pos)) 
    return mylst

#find_segments_str2(dji.rawdata)
#pos=4

from collections import OrderedDict
def segments(data):
    seg = OrderedDict()
    nextpos = 0
    while True:
        currentpos = nextpos
        nextpos = data.find(b"\xFF",nextpos)
        if nextpos == -1: break
        marker = data[nextpos:nextpos+2]
        if marker in MARKER.keys():
            name = MARKER[marker]
            length = data[nextpos+2:nextpos+4]
            nextpos = nextpos + 256 * length[0] + length[1] + 2
            seg[currentpos] = {"pos":currentpos,"name":name,"end":nextpos}     
        elif marker in MARKER_2BYTE.keys():
            name = MARKER_2BYTE[marker]
            nextpos = nextpos + 2
            seg[currentpos] = {"pos":currentpos,"name":name,"end":nextpos}     
        else:
            nextpos = nextpos + 1
            if marker == b"\xff\xda": # Start of Scan has not a known length
                seg[currentpos] = {"pos":currentpos,"name":"SOS","end":nextpos}     
    return seg


j = JFIF()

def test_segments_class():
    for i in filelist:
        with open(i, mode="rb") as f:
            rawdata = f.read()
            j.__find_segments__(rawdata)
            
def test_segments_func():
    for i in filelist:
        with open(i, mode="rb") as f:
            rawdata = f.read()
            find_segments_re(rawdata)

def test_segments_str():
    for i in filelist:
        with open(i, mode="rb") as f:
            rawdata = f.read()
            find_segments_str(rawdata)

def test_segments_str2():
    for i in filelist:
        with open(i, mode="rb") as f:
            rawdata = f.read()
            find_segments_str2(rawdata)

def test_segments():
    for i in filelist:
        #print (i)
        with open(i, mode="rb") as f:
            rawdata = f.read()
            #print(len(rawdata))
            find_main_segments(rawdata)


def test_segments2():
    for i in filelist:
        #print (i)
        with open(i, mode="rb") as f:
            rawdata = f.read()
            #print(len(rawdata))
            segments(rawdata)

    
filepath = "G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras/FLIR/FLIR_Vue_640_20200517_110530/20200517_110530_R.jpg"                
with open(filepath, mode="rb") as f:
    rawdata = f.read()
print(len(rawdata))
seg = find_segments_re(rawdata)
for s in seg:
    print(s["id"],"\t",s["segment"],"\t",s["pos"],"\t",s["end"],"\t",s["parent"],"\t",s["level"],"\t",s["type"])


704479
0 	 SOI 	 0 	 2 	 0 	 0 	 
1 	 APP0 	 2 	 20 	 0 	 0 	 b'JFIF'
2 	 APP1 	 20 	 2258 	 0 	 0 	 b'Exif'
3 	 APP8 	 2171 	 2941 	 2 	 1 	 
4 	 APP1 	 2258 	 67794 	 3 	 2 	 b'FLIR'
5 	 TEM 	 2798 	 2800 	 4 	 3 	 
6 	 APP1 	 67794 	 133330 	 0 	 0 	 b'FLIR'
7 	 APP1 	 133330 	 198866 	 0 	 0 	 b'FLIR'
8 	 APP1 	 198866 	 264402 	 0 	 0 	 b'FLIR'
9 	 APP1 	 264402 	 329938 	 0 	 0 	 b'FLIR'
10 	 APP1 	 329938 	 395474 	 0 	 0 	 b'FLIR'
11 	 APP1 	 395474 	 461010 	 0 	 0 	 b'FLIR'
12 	 APP1 	 461010 	 526546 	 0 	 0 	 b'FLIR'
13 	 APP1 	 526546 	 592082 	 0 	 0 	 b'FLIR'
14 	 APP1 	 592082 	 657618 	 0 	 0 	 b'FLIR'
15 	 APP1 	 657618 	 661738 	 0 	 0 	 b'FLIR'
16 	 TEM 	 658310 	 658312 	 15 	 1 	 
17 	 DQT 	 659073 	 661954 	 15 	 1 	 
18 	 DQT 	 661738 	 661872 	 17 	 2 	 
19 	 DHT 	 661872 	 662292 	 17 	 2 	 
20 	 SOF0 	 662292 	 662311 	 0 	 0 	 
21 	 DRI 	 662311 	 662317 	 0 	 0 	 
22 	 RST0 	 665323 	 729176 	 0 	 0 	 
23 	 RST1 	 668213 	 729700 	 22 	 1 	 
24 	 RST2 	 671

In [7]:
filelist
test_segments2()


In [8]:
print(timeit.timeit("test_segments()", globals=locals(), number=10))


0.2662621999916155


In [244]:
print(timeit.timeit("test_segments2()", globals=locals(), number=10))

0.3200902999997197


In [201]:
print(timeit.timeit("test_segments_class()", globals=locals(), number=10))

0.7282018999994762


In [202]:
print(timeit.timeit("test_segments_func()", globals=locals(), number=10))

0.709334500001205


In [203]:
print(timeit.timeit("test_segments_str()", globals=locals(), number=10))

0.6872048000004725


In [204]:
print(timeit.timeit("test_segments_str2()", globals=locals(), number=10))

0.31879709999884653


In [205]:
print(timeit.timeit("test_segments_str3()", globals=locals(), number=10))

C:/WILDRETTER DATEN/2020_Easy/700101_0000_Fotosession2\200527_082125_9.jpg
C:/WILDRETTER DATEN/2020_Easy/700101_0000_Fotosession2\200527_082125_9.jpg
C:/WILDRETTER DATEN/2020_Easy/700101_0000_Fotosession2\200527_082125_9.jpg
C:/WILDRETTER DATEN/2020_Easy/700101_0000_Fotosession2\200527_082125_9.jpg
C:/WILDRETTER DATEN/2020_Easy/700101_0000_Fotosession2\200527_082125_9.jpg
C:/WILDRETTER DATEN/2020_Easy/700101_0000_Fotosession2\200527_082125_9.jpg
C:/WILDRETTER DATEN/2020_Easy/700101_0000_Fotosession2\200527_082125_9.jpg
C:/WILDRETTER DATEN/2020_Easy/700101_0000_Fotosession2\200527_082125_9.jpg
C:/WILDRETTER DATEN/2020_Easy/700101_0000_Fotosession2\200527_082125_9.jpg
C:/WILDRETTER DATEN/2020_Easy/700101_0000_Fotosession2\200527_082125_9.jpg
0.013704500001040287


In [206]:
print(timeit.timeit("test_regular()", globals=locals(), number=10))

0.24313450000045123


In [193]:
print(timeit.timeit("test_mmap()", globals=locals(), number=10))

5.426825800001097


In [19]:
import mmap

def mmap_io(filename):
    with open(filename, mode="rb") as file_obj:
        with mmap.mmap(file_obj.fileno(), length=0, access=mmap.ACCESS_READ) as mmap_obj:
            text = mmap_obj.read()
#            print(text)

def regular_io(filename):
    with open(filename, mode="rb") as file_obj:
        text = file_obj.read()
 #       print(text)

def test_mmap():
    for i in filelist:
        mmap_io(i)
#test_mmap
def test_regular():
    for i in filelist:
        regular_io(i)
test_regular()

# EXIFREAD

In [33]:
tdpath = "G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras/thermalDRONES/FWR2020_Boson320/200509_140849_7.jpg"
djipath = "G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras/DJI/Testmaterial Mavic 2 Enterprise Advanced/Hausen_2021-08/DJI_0008.JPG"
with open(tdpath,"rb") as f:
    td = exifread.process_file(f,details=False)
with open(djipath,"rb") as f:
    dji = exifread.process_file(f,details=False)
#dji["EXIF DateTimeOriginal"].values#.field_length
#td["EXIF DateTimeOriginal"].values#.field_length
dir(dji["EXIF DateTimeOriginal"])
#dji["EXIF DateTimeOriginal"].field_offset

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'field_length',
 'field_offset',
 'field_type',
 'printable',
 'tag',
 'values']

In [5]:
import exifread

def testexifread():
    for i in filelist:
        with open(i,"rb") as f:
            exif = exifread.process_file(f,details=False)

with open(filelist[0],"rb") as f:
    exif = exifread.process_file(f,details=False)
print (exif)

{'Image BitsPerSample': (0x0102) Short=8 @ 18, 'Image Make': (0x010F) ASCII=FLIR @ 170, 'Image Model': (0x0110) ASCII=Boson @ 175, 'Image XResolution': (0x011A) Ratio=72 @ 181, 'Image YResolution': (0x011B) Ratio=72 @ 189, 'Image ResolutionUnit': (0x0128) Short=Pixels/Inch @ 78, 'Image YCbCrPositioning': (0x0213) Short=Centered @ 90, 'Image FocalLength': (0x920A) Ratio=14 @ 197, 'Image FNumber': (0x829D) Ratio=1 @ 205, 'Image ExifOffset': (0x8769) Long=213 @ 126, 'GPS GPSVersionID': (0x0000) Byte=[3, 2] @ 2119, 'GPS GPSLatitudeRef': (0x0001) ASCII=N @ 2131, 'GPS GPSLatitude': (0x0002) Signed Ratio=[48, 8, 245847/5000] @ 2271, 'GPS GPSLongitudeRef': (0x0003) ASCII=E @ 2155, 'GPS GPSLongitude': (0x0004) Signed Ratio=[11, 0, 6471/1250] @ 2295, 'GPS GPSAltitudeRef': (0x0005) Byte=0 @ 2179, 'GPS GPSAltitude': (0x0006) Signed Ratio=56799/100 @ 2319, 'GPS GPSTimeStamp': (0x0007) ASCII=2020:05:27 @ 2351, 'GPS GPSSpeedRef': (0x000C) ASCII=K @ 2227, 'GPS GPSSpeed': (0x000D) Ratio=0 @ 2361, 'GPS 

# PIL

In [6]:
from PIL import Image as PILImage

def testPIL():
    for i in filelist:
        img = PILImage.open(i)
        exif = img._getexif()
            
img = PILImage.open(filelist[0])
info = img._getexif()
print (info)        


{36864: b'0210', 37121: b'\x01\x02\x03', 40960: b'0100', 40961: 1, 36867: '2020:05:27 08:21:25', 40962: 320, 40963: 256, 258: 8, 37386: 14.0, 271: 'FLIR', 272: 'Boson', 37521: '980973', 37393: 7, 531: 1, 282: 72.0, 283: 72.0, 33437: 1.0, 34853: {0: b'\x03\x02', 1: 'N', 2: (48.0, 8.0, 49.1694), 3: 'E', 4: (11.0, 0.0, 5.1768), 5: b'\x00', 6: 567.99, 7: '2020:05:27', 12: 'K', 13: 0.0, 14: 'T', 15: 0.0}, 296: 2, 50735: 'S', 700: b'<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\r\n            xmlns:Camera="http://pix4d.com/camera/1.0/"\r\nxmlns:FLIR="http://ns.flir.com/xmp/1.0/">\r\n            <rdf:Description rdf:about="">\r\n            <Camera:BandName>\r\n<rdf:Seq>\r\n<rdf:li>LWIR</rdf:li>\r\n</rdf:Seq>\r\n</Camera:BandName>\r\n            <Camera:CentralWavelength>\r\n<rdf:Seq>\r\n<rdf:li>10000</rdf:li>\r\n</rdf:Seq>\r\n</Camera:CentralWavelength>\r\n            <Camera:WavelengthFWHM>\r\n<rdf:Seq>\r\n<rdf:li>4500</rdf:li>\r\n</rdf:Seq>\r\n</Camera:WavelengthFWHM>\r\

In [7]:
from io import BytesIO
def image_from_buffer_PIL(buf):
    im = PILImage.open(BytesIO(buf))
    return im



In [8]:
exif,xmp,rawdata = get_exif_xmp("G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras/thermalDRONES/Grabber_9.0_FWR2020Herelink_640/210514_130432_1.jpg")
#image_from_buffer_PIL(rawdata)


NameError: name 'get_exif_xmp' is not defined

In [180]:
def test_piexif_pil_xml2dict():
    for i in filelist:
        xmp = ""
        rawdata = []
        with open(i,"rb") as f:
            rawdata = f.read()
            exif= piexif.load(rawdata)
        xmp_start = rawdata.lower().find(b"<rdf:rdf")
        xmp_end = rawdata.lower().find(b"</rdf:rdf")
        xmp_str = rawdata[xmp_start:xmp_end+10]
        xmp = xmltodict.parse(xmp_str)
        rdf = xmp.get("rdf:RDF").get("rdf:Description")
        img = image_from_buffer_PIL(rawdata)

# PIEXIF

In [102]:
import piexif

def test_piexif():
    for i in filelist:
        exif = piexif.load(i)
        
exif = piexif.load(filelist[0])
print (exif)            

{'0th': {258: 8, 271: b'FLIR', 272: b'Boson', 282: (72, 1), 283: (72, 1), 296: 2, 531: 1, 34665: 213, 34853: 2109, 50735: b'', 37393: 7}, 'Exif': {36864: b'0210', 37121: b'\x01\x02\x03', 40960: b'0100', 40961: 1, 36867: b'2020:05:27 08:21:2', 37521: b'98097', 40962: 320, 40963: 256}, 'GPS': {0: (3, 2), 1: b'', 2: ((48, 1), (8, 1), (245847, 5000)), 3: b'', 4: ((11, 1), (0, 1), (6471, 1250)), 5: 0, 6: (56799, 100), 7: b'2020:05:2', 12: b'', 13: (0, 1000), 14: b'', 15: (0, 100)}, 'Interop': {}, '1st': {}, 'thumbnail': None}


In [137]:
path = "G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras"
p = os.path.join(path,"DJI/DJI-Farbkamera_neu/DJI_0001.JPG")
from bs4 import BeautifulSoup
def test_piexif_bs():
    for i in filelist:
        xmp = ""
        d = []
        with open(i,"rb") as f:
            d = f.read()
            exif= piexif.load(d)
        xmp_start = d.lower().find(b"<rdf:rdf")
        xmp_end = d.lower().find(b"</rdf:rdf")
        xmp_str = d[xmp_start:xmp_end+10]
        xmp = BeautifulSoup(xmp_str,"lxml")
        

In [138]:
def test_piexif_xml2dict():
    for i in filelist:
        xmp = ""
        d = []
        with open(i,"rb") as f:
            d = f.read()
            exif= piexif.load(d)
        xmp_start = d.lower().find(b"<rdf:rdf")
        xmp_end = d.lower().find(b"</rdf:rdf")
        xmp_str = d[xmp_start:xmp_end+10]
        xmp = xmltodict.parse(xmp_str)
        rdf = xmp.get("rdf:RDF").get("rdf:Description")


In [107]:
def get_exif_xmp(filepath):
    xmp = ""
    d = []
    with open(filepath,"rb") as f:
        d = f.read()
        exif= piexif.load(d)
    xmp_start = d.lower().find(b"<rdf:rdf")
    xmp_end = d.lower().find(b"</rdf:rdf")
    xmp_str = d[xmp_start:xmp_end+10]
    xmp = xmltodict.parse(xmp_str)
    rdf = xmp.get("rdf:RDF").get("rdf:Description")
    return exif, rdf

In [131]:
def get_exif_xmp(filepath):
        with open(filepath,"rb") as f:
            rawdata = f.read()
            exif= piexif.load(rawdata)
        xmp_start = d.lower().find(b"<rdf:rdf")
        xmp_end = d.lower().find(b"</rdf:rdf")
        xmp_str = d[xmp_start:xmp_end+10]
        xmp_od = xmltodict.parse(xmp_str)
        xmp = xmp_od.get("rdf:RDF").get("rdf:Description")
        return exif, xmp, rawdata 

## openCV2

In [129]:
import cv2
def test_cv2():
    for i in filelist:
        img = cv2.imread(i)
        
img = cv2.imread(filelist[0])
print (img.shape)            

(256, 320, 3)


In [195]:
import numpy as np
def image_from_buffer_cv2(buf):
    im = cv2.imdecode(np.frombuffer(buf, np.uint8), 1)
    return im
#image_from_buffer_cv2(rawdata)    

def test_piexif_cv2_xml2dict():
    for i in filelist:
        xmp = ""
        rawdata = []
        with open(i,"rb") as f:
            rawdata = f.read()
            exif= piexif.load(rawdata)
        xmp_start = rawdata.lower().find(b"<rdf:rdf")
        xmp_end = rawdata.lower().find(b"</rdf:rdf")
        xmp_str = rawdata[xmp_start:xmp_end+10]
        xmp = xmltodict.parse(xmp_str)
        rdf = xmp.get("rdf:RDF").get("rdf:Description")
        img = image_from_buffer_cv2(rawdata)

# TESTS


## Compare Exif read

In [133]:
#Lib exifread
print(timeit.timeit("testexifread()", globals=locals(), number=10))

1.125467399993795


In [155]:
#Lib piexif
print(timeit.timeit("test_piexif()", globals=locals(), number=10))

0.5870674000034342


In [156]:
#Lib piexif with beautiful soup
print(timeit.timeit("test_piexif_bs()", globals=locals(), number=10))

3.9065226999955485


In [157]:
#Lib piexif with xml2dict
print(timeit.timeit("test_piexif_xml2dict()", globals=locals(), number=10))

1.4698589000036009


## Compare Jpeg read to numpy

In [158]:
#Lib PIL
print(timeit.timeit("testPIL()", globals=locals(), number=10))

1.6396391000016592


In [159]:
#Lib openCV2
print(timeit.timeit("test_cv2()", globals=locals(), number=10))

1.6535896999994293


## Compare read header with read header and image PIL or cv2

In [185]:
print(timeit.timeit("test_piexif_xml2dict()", globals=locals(), number=10))

1.4195973000023514


In [197]:
print(timeit.timeit("test_piexif_pil_xml2dict()", globals=locals(), number=10))

2.3099307999946177


In [198]:
print(timeit.timeit("test_piexif_cv2_xml2dict()", globals=locals(), number=10))

2.6766009999992093


# for loop vs list comprehension

In [14]:
def long_for_loop():
    out = []
    for i in filelist:
        out.append(i.strip("\x00"))
def long_list_comprehension():
    out = [i.strip("\x00") for i in filelist]

In [30]:
print(timeit.timeit("long_list_comprehension()", globals=locals(), number=10000))

0.1702205000037793


In [31]:
print(timeit.timeit("long_for_loop()", globals=locals(), number=10000))

0.3293652000138536


# test on different Cameras

In [112]:
for root, folders,files in os.walk("G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras"):
    print(root)
    i = 0
    for f in files:
        i = i + 1
        if i >5: break
        base, ext = os.path.splitext(f)
        if ext.lower() not in [".jpg",".jpeg",".ara",".raw",".tif",".tiff"]: continue
        fp = os.path.join(root,f)
        try:
            exif, rdf = get_exif_xmp(fp)
            #exif = piexif.load(fp)
            print("\t",f, exif['0th'][271], len(dict(rdf)))
        except KeyError:
            print("\t",f, exif['0th'], len(dict(rdf)))
        except:
            print ("\t",f, "Exif read failed")

G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras
G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\yuneec
G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\yuneec\CGOET, Hot White
	 CGOET-HotWhite-100m.jpeg b'YUNEE' 21
	 CGOET-HotWhite-10m.jpeg b'YUNEE' 21
	 CGOET-HotWhite-30m.jpeg b'YUNEE' 21
	 CGOET-HotWhite-50m.jpeg b'YUNEE' 21
G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\yuneec\CGOET, Rainbow
	 CGOET-Rainbow-100m.jpeg b'YUNEE' 21
	 CGOET-Rainbow-10m.jpeg b'YUNEE' 21
	 CGOET-Rainbow-30m.jpeg b'YUNEE' 21
	 CGOET-Rainbow-50m.jpeg b'YUNEE' 21
	 CGOET-Rainbow-80m.jpeg b'YUNEE' 21
G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\yuneec\cp2102-driver
G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\yuneec\cp2102-driver\x64
G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\yuneec\cp2102-dr

	 20200517_110534_R.jpg b'FLIR\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' 24
	 20200517_110536_R.jpg b'FLIR\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' 24
	 20200517_110538_R.jpg b'FLIR\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00' 24
G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\FLIR\Flir_VuePro
	 vuepro_example.jpg Exif read failed
	 vuepro_example.tiff Exif read failed
	 vuepro_example2.tiff Exif read failed
G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\Canon
G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\Canon\EOS 100D
	 IMG_7890_geotag.JPG b'Canon' 2
G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\Parrot
G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras\Parrot\AnafiUSA_320pix_50deg
	 P0180018.JPG b'Parrot' 73
	 P0140014.JPG b'Parrot' 

In [12]:
filepath = "G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras/DJI/Mavic2Pro/DJI_0161.jpg"
with open(filepath,"rb") as f:
    exif = exifread.process_file(f,details=False)
print (exif)

{'Image XResolution': (0x011A) Ratio=240 @ 50, 'Image YResolution': (0x011B) Ratio=240 @ 58, 'Image ResolutionUnit': (0x0128) Short=Pixels/Inch @ 42, 'Thumbnail Compression': (0x0103) Short=JPEG (old-style) @ 76, 'Thumbnail XResolution': (0x011A) Ratio=72 @ 144, 'Thumbnail YResolution': (0x011B) Ratio=72 @ 152, 'Thumbnail ResolutionUnit': (0x0128) Short=Pixels/Inch @ 112, 'Thumbnail JPEGInterchangeFormat': (0x0201) Long=160 @ 124, 'Thumbnail JPEGInterchangeFormatLength': (0x0202) Long=11565 @ 136}


In [13]:
path = "G:/Geteilte Ablagen/thermal DRONES GmbH/Wildretter-Daten/verschiedene Kameras"
p = os.path.join(path,"DJI/DJI-Farbkamera_neu/DJI_0001.JPG")
with open(p,"rb") as f:
    d = f.read()
    ifd = piexif.load(d)
#ifd

In [14]:
ifd

{'0th': {256: 5472,
  257: 3648,
  258: (8, 8, 8),
  270: b'default',
  271: b'Hasselblad',
  272: b'L1D-20c',
  274: 1,
  277: 3,
  282: (72, 1),
  283: (72, 1),
  296: 2,
  305: b'10.00.11.04',
  306: b'2020:05:14 13:41:47',
  531: 1,
  34665: 332,
  34853: 746,
  40092: (48,
   0,
   46,
   0,
   57,
   0,
   46,
   0,
   49,
   0,
   52,
   0,
   50,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
 

# XMl PARSER

In [15]:
#print(timeit.timeit("test()", setup="from __main__ import test"))

# For Python>=3.5 one can also write:
xmp_str = test_piexif2()

In [16]:
xmp = BeautifulSoup(xmp_str,"lxml")
xmp_str

b'<rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"\r\n            xmlns:Camera="http://pix4d.com/camera/1.0/"\r\nxmlns:FLIR="http://ns.flir.com/xmp/1.0/">\r\n            <rdf:Description rdf:about="">\r\n            <Camera:BandName>\r\n<rdf:Seq>\r\n<rdf:li>LWIR</rdf:li>\r\n</rdf:Seq>\r\n</Camera:BandName>\r\n            <Camera:CentralWavelength>\r\n<rdf:Seq>\r\n<rdf:li>10000</rdf:li>\r\n</rdf:Seq>\r\n</Camera:CentralWavelength>\r\n            <Camera:WavelengthFWHM>\r\n<rdf:Seq>\r\n<rdf:li>4500</rdf:li>\r\n</rdf:Seq>\r\n</Camera:WavelengthFWHM>\r\n            <Camera:TlinearGain>0.00</Camera:TlinearGain>\r\n            <Camera:Yaw>0/100</Camera:Yaw>\r\n            <Camera:Pitch>-90.0</Camera:Pitch>\r\n            <Camera:Roll>0/100</Camera:Roll>\r\n            <Camera:GPSXYAccuracy>0</Camera:GPSXYAccuracy>\r\n            <Camera:GPSZAccuracy>0</Camera:GPSZAccuracy>\r\n            <Camera:GyroRate>0.00</Camera:GyroRate>\r\n            <Camera:DetectorBitDepth>16</Camer

In [17]:
import xmltodict


In [66]:
xmp = xmltodict.parse(xmp_str)

rdf = xmp["rdf:RDF"]["rdf:Description"]

for i in rdf.items():
    print(i[0],i[1])

@rdf:about 
Camera:BandName OrderedDict([('rdf:Seq', OrderedDict([('rdf:li', 'LWIR')]))])
Camera:CentralWavelength OrderedDict([('rdf:Seq', OrderedDict([('rdf:li', '10000')]))])
Camera:WavelengthFWHM OrderedDict([('rdf:Seq', OrderedDict([('rdf:li', '4500')]))])
Camera:TlinearGain 0.00
Camera:Yaw 0/100
Camera:Pitch -90.0
Camera:Roll 0/100
Camera:GPSXYAccuracy 0
Camera:GPSZAccuracy 0
Camera:GyroRate 0.00
Camera:DetectorBitDepth 16
Camera:IsNormalized 1
Camera:SerialNumber S
FLIR:MAVVersionID 0.3.0.0
FLIR:MAVComponentID 100
FLIR:MAVRelativeAltitude 0.463
FLIR:MAVRateOfClimbRef M
FLIR:MAVRateOfClimb 0/1000
FLIR:MAVYaw 173.51
FLIR:MAVPitch 0/100
FLIR:MAVRoll 0/100
FLIR:MAVYawRate 0/100
FLIR:MAVPitchRate 0/100
FLIR:MAVRollRate 0/100


In [91]:
dict(rdf)["Camera:Yaw"]

'0/100'

In [59]:
od = list(xmp.items())
list(list(od[0][1].items())[3][1].items())


[('@rdf:about', ''),
 ('Camera:BandName',
  OrderedDict([('rdf:Seq', OrderedDict([('rdf:li', 'LWIR')]))])),
 ('Camera:CentralWavelength',
  OrderedDict([('rdf:Seq', OrderedDict([('rdf:li', '10000')]))])),
 ('Camera:WavelengthFWHM',
  OrderedDict([('rdf:Seq', OrderedDict([('rdf:li', '4500')]))])),
 ('Camera:TlinearGain', '0.00'),
 ('Camera:Yaw', '0/100'),
 ('Camera:Pitch', '-90.0'),
 ('Camera:Roll', '0/100'),
 ('Camera:GPSXYAccuracy', '0'),
 ('Camera:GPSZAccuracy', '0'),
 ('Camera:GyroRate', '0.00'),
 ('Camera:DetectorBitDepth', '16'),
 ('Camera:IsNormalized', '1'),
 ('Camera:SerialNumber', 'S'),
 ('FLIR:MAVVersionID', '0.3.0.0'),
 ('FLIR:MAVComponentID', '100'),
 ('FLIR:MAVRelativeAltitude', '0.463'),
 ('FLIR:MAVRateOfClimbRef', 'M'),
 ('FLIR:MAVRateOfClimb', '0/1000'),
 ('FLIR:MAVYaw', '173.51'),
 ('FLIR:MAVPitch', '0/100'),
 ('FLIR:MAVRoll', '0/100'),
 ('FLIR:MAVYawRate', '0/100'),
 ('FLIR:MAVPitchRate', '0/100'),
 ('FLIR:MAVRollRate', '0/100')]